In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely
from shapely.geometry import Point
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import json
import re
import openpyxl
import seaborn as sns
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import random
from pykml import parser
from os import path

In [3]:
path = '/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/GIS_Polimi/vento/griglia_vento.shp'
df = gpd.read_file(path)
df.head()
df['Id'] = df['Id'].astype(str)
path_scores = '/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/csv_files/scores_conditioned.csv'
scores = pd.read_csv(path_scores, sep = ',')
scores['Id'] = scores['Id'].astype(str)
scores.rename(columns = {'ID' : 'Id', 'Unnamed: 1': 'PC1', 'Unnamed: 2': 'PC2'}, inplace = True)
final_geom = df.merge(scores, how = 'left', on = 'Id')
final_geom['lon'] = final_geom['geometry'].apply(lambda x : np.dstack((x.exterior.coords.xy)).tolist()[0][0][0] )
final_geom['lat'] = final_geom['geometry'].apply(lambda x : np.dstack((x.exterior.coords.xy)).tolist()[0][0][1] )
mask = (final_geom.lat >= 44.89) & (final_geom.lon >= 9.16 )
final_geom = final_geom[~mask]
final_geom.drop(['lat', 'lon'], axis = 1, inplace = True)
final_geom = final_geom.to_crs("EPSG:4326")

final_geom.to_file('/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/GIS_Polimi/projected/vento/original_crs_poly.shp')

final_geom = final_geom.to_crs("EPSG:32632")
final_geom.Shape_Area = final_geom.geometry.area
final_geom.Shape_Leng = final_geom.geometry.length
final_geom.to_file('/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/GIS_Polimi/projected/vento/projected_poly.shp')
